In [10]:
import pandas as pd

In [28]:
#import mine data
#primary data
mine_pri = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-April-2024 (1).xlsx", sheet_name="Global Coal Mine Tracker (Non-C")
#supplement data
mine_sup1 = pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-September-2024-Supplement-v2 (2).xlsx", sheet_name="Historical Production (non-Chin")
mine_sup2 =pd.read_excel("./Data/new coal/Global-Coal-Mine-Tracker-December-2024-Supplement-Historical-Production-from-2018-to-2023.xlsx", sheet_name="Historical Production(non-China")

In [30]:
#import plant data
plant = pd.read_excel("./Data/new coal/Global-Coal-Plant-Tracker-January-2025.xlsx", sheet_name="Units")

#correct the estimated capacity with capacity factor
plant['Est_capacity (MW)']=plant['Capacity (MW)']*plant['Capacity factor']

#import terminal data
#note: the capacity of terminal dataset may be overestimated, because some terminals not only receive 'coal' but 'all cargo'
terminal = pd.read_excel("./Data/new coal/Global-Coal-Terminals-Tracker-December-2024.xlsx", sheet_name="Terminals")

In [16]:
# Step 1: Drop columns
#def drop_columns(df):
    #columns_to_keep = ['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)', 
                       #'Year of Production',
                       'Primary Consumer, Destination', 'Coal Plant, Steel Plant, Terminal', 'Owners', 'Parent Company']
    #columns_to_drop = set(df.columns) - set(columns_to_keep)
   # return df.drop(columns=columns_to_drop)


#drop_columns(mine_pri)

IndentationError: unexpected indent (2013213400.py, line 5)

In [31]:
#may further filter by 'status == 'operating'
mine_pri = mine_pri[mine_pri['Country'] == 'India']
plant = plant[plant['Country/Area'] == 'India']
terminal = terminal[terminal['Country/Area'] == 'India']

## Sheet 1: Assets

In [38]:
mine_assets = mine_pri[['GEM Mine ID', 'Mine Name', 'Latitude', 'Longitude', 'Country', 'Production (Mtpa)']]
plant_assets = plant[['GEM location ID', 'Plant name', 'Latitude', 'Longitude','Country/Area', 'Est_capacity (MW)' ]]
terminal_assets = terminal[['GEM Terminal ID', 'Coal Terminal Name', 'Latitude', 'Longitude','Country/Area', 'Capacity (Mt)']]

mine_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
plant_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']
terminal_assets.columns = ['asset_id', 'name', 'latitude', 'longitude', 'country', 'production']

mine_assets['product'] = 'coal'
plant_assets['product'] = 'electricity'
terminal_assets['product'] = 'coal'#may be incorrect because some are 'all cargo'

def process_assets(df):
    # 拆分数据：production = '*' 的行 和 其余可计算的行
    star_rows = df[df["production"] == '*']  # 只包含 '*' 生产值的行
    numeric_rows = df[df["production"] != '*'].copy()  # 其余行可进行数值合并

    # 确保 production 是数值类型
    numeric_rows["production"] = pd.to_numeric(numeric_rows["production"], errors="coerce").fillna(0)

    # 仅对数值部分进行 groupby 聚合
    grouped_numeric = numeric_rows.groupby("asset_id", as_index=False).agg({
        "name": "first",
        "latitude": "first",
        "longitude": "first",
        "country": "first",
        "production": "sum",
        "product": "first"
    })

    # 合并 groupby 结果 + production = '*' 的原始行
    return pd.concat([grouped_numeric, star_rows], ignore_index=True)

# 处理各个数据集
mine_assets = process_assets(mine_assets)
plant_assets = process_assets(plant_assets)
terminal_assets = process_assets(terminal_assets)

# 合并所有资产数据
total_assets = pd.concat([mine_assets, plant_assets, terminal_assets], ignore_index=True)

# 输出最终结果
total_assets

/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_845/1934385193.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mine_assets['product'] = 'coal'
/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_845/1934385193.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  plant_assets['product'] = 'electricity'
/var/folders/wy/k_hfp9ls0msf61dg9qbm3h040000gn/T/ipykernel_845/1934385193.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

,asset_id,name,latitude,longitude,country,production,product
0,M0464,Adasa Coal Mine,21.347003,78.953545,India,0.61,coal
1,M0465,Adriyala Coal Mine,18.6648,79.579,India,1.54,coal
2,M0466,AKK Coal Mine,23.7905,85.9198,India,4.25,coal
3,M0467,Amadand Coal Mine,23.157942,82.061533,India,0.87,coal
4,M0468,Amalgamated Keshalpur-West Mudidih (AKWMC) Coa...,23.8078,86.3221,India,4.1,coal
...,...,...,...,...,...,...,...
1243,T1415,Visakhapatnam Port Adani Terminal,17.68333,83.2833,India,6.4,coal
1244,T1432,Rewas Port Coal Terminal,18.79178,72.914995,India,66.0,coal
1245,T1383,Adani Hazira Port Coal Terminal,21.08234,72.627231,India,*,coal
1246,T1019,Dahanu Port Terminal,19.97,72.73,India,*,coal


In [47]:
prod_index = total_assets.columns.get_loc('production')

total_assets.insert(prod_index, 'replacement_value', '')

ValueError: cannot insert replacement_value, already exists

## Sheet 2: Firms

In [54]:
import numpy as np

def process_firms_data(df, owner_column, firm_type):
    """
    通用函数处理 firms 数据

    参数：
    df: DataFrame - 原始数据
    owner_column: str - 所有者列名
    firm_type: str - 企业类型 ('production' 等)

    返回：
    DataFrame - 处理后的数据
    """
    firms = df[[owner_column]].rename(columns={owner_column: 'name'})
    
    # 添加必要的列，初始化为空值
    firms['firm_id'] = np.nan
    firms['final_demand'] = np.nan
    firms['discount_rate'] = np.nan
    firms['unit_price'] = np.nan
    firms['margin'] = np.nan
    firms['leverage_ratio'] = np.nan
    firms['type'] = firm_type  # 设定企业类型
    
    # 调整列顺序
    firms = firms[['firm_id', 'name', 'final_demand', 'discount_rate', 
                   'unit_price', 'margin', 'leverage_ratio', 'type']]
    
    return firms

# 处理三个数据集
mine_firms = process_firms_data(mine_pri, 'Owners', 'production')
plant_firms = process_firms_data(plant, 'Owner', 'production')
terminal_firms = process_firms_data(terminal, 'Owner', 'production')
total_firms = pd.concat([mine_firms, plant_firms, terminal_firms], ignore_index=True)
total_firms.drop_duplicates(subset=['name'], keep='first', inplace=True)

with pd.ExcelWriter('./Data/coal_supply_chain_india.xlsx') as writer:
    total_assets.to_excel(writer, sheet_name='assets', index=False)
    total_firms.to_excel(writer, sheet_name='firms', index=False)

In [55]:
total_firms

,firm_id,name,final_demand,discount_rate,unit_price,margin,leverage_ratio,type
0,NaN,Western Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
1,NaN,The Singareni Collieries Co Ltd,NaN,NaN,NaN,NaN,NaN,production
2,NaN,Central Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
3,NaN,South Eastern Coalfields Ltd,NaN,NaN,NaN,NaN,NaN,production
5,NaN,Bharat Coking Coal Ltd,NaN,NaN,NaN,NaN,NaN,production
...,...,...,...,...,...,...,...,...
2548,NaN,Shree Cement,NaN,NaN,NaN,NaN,NaN,production
2549,NaN,Tata Group,NaN,NaN,NaN,NaN,NaN,production
2552,NaN,V.O. Chidambaranar Port Trust,NaN,NaN,NaN,NaN,NaN,production
2555,NaN,Visakhapatnam Port Trust (VPT),NaN,NaN,NaN,NaN,NaN,production
